<h2><center><font color="black"><strong>OLS pour les appartements à Paris</strong></font></center></h2>

In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [115]:
import statsmodels.api as sm
import math
from sklearn.metrics import mean_squared_error

In [181]:
df = pd.read_csv('../Data/update_data/df_Paris_Processed_Michel.csv', low_memory = False, encoding = 'latin1')

In [214]:
%run '../Helpers/preprocessing.py'
%run '../Helpers/graph.py'
%run '../Helpers/score.py'
%run '../Helpers/model_implementation.py'

In [183]:
    # Créer les variables en log
log_var(df, ['Prix_m2'])
log_var(df, ['valeur_fonciere'])
log_var(df, ['surface_reelle_bati'])
    # Ajouter la variable ANNEE-TRIMESTRE
annee_trimestre(df)

## Preprocessing

Ajouter des dummies pour les variables continues lorsqu'elles dépassent un certain seuil.

In [184]:
dummies_pr_var_continues(df, 'Nombre_dependance', 0)

In [185]:
dummies_pr_var_continues(df, 'nombre_pieces_principales', 2)

Ajouter des dummies pour les variables catégorielles.

In [186]:
def dummies_pr_var_cat(data, var) : 
    data[var] = data[var].astype(str)
    nom_tab = var + '_dummies'
    nom_tab = pd.get_dummies(data=data[[var]], drop_first=True)
    
    nom_col = list(nom_tab.columns)
    
    return(data.merge(nom_tab, how = 'left', right_index = True, left_index = True), nom_col)

In [187]:
df, col_trimestre = dummies_pr_var_cat(df, 'num_trimestre')

In [188]:
df, col_iris = dummies_pr_var_cat(df, 'iris_code')

In [189]:
df, col_arrondissement = dummies_pr_var_cat(df, 'Arrondissement')

In [190]:
df, col_month = dummies_pr_var_cat(df, 'Month')

In [191]:
#df = df.merge(tab, how = 'left', right_index = True, left_index = True)
df.head(3)

,Unnamed: 0,id_mutation,date_mutation,code_type_local,code_commune,surface_terrain,surface_reelle_bati,nombre_pieces_principales,nature_culture,valeur_fonciere,latitude,longitude,nombre_lots,numero_disposition,code_departement,Nombre_house,Nombre_dependance,geometry,iris_code,iris_name,Prix_m2,Year,Month,prix_m2_moy_surf_habit,prix_m2_moy_terrain,Vente_par_iris,num_trimestre,quarter,IPL,IPL_{Q-1},N_logements,N_res_ppale,N_res_second,N_vacant,N_maisons,N_apparts,N_proprietaire,N_locataire,Arrondissement,n_metros_iris,dist_metro,trimestre,prix_m2_moy_IRIS_trimestri_surf_habit,taux_accroissement,Vente_par_iris_tri,Prix_m2_log,valeur_fonciere_log,surface_reelle_bati_log,Nombre_dependance_dummy,nombre_pieces_principales_dummy,num_trimestre_2,num_trimestre_3,num_trimestre_4,iris_code_751010102,iris_code_751010103,iris_code_751010201,iris_code_751010202,iris_code_751010203,iris_code_751010204,iris_code_751010206,iris_code_751010301,iris_code_751010302,iris_code_751010303,iris_code_751010401,iris_code_751010402,iris_code_751020501,iris_code_751020502,iris_code_751020503,iris_code_751020601,iris_code_751020602,iris_code_751020701,iris_code_751020702,iris_code_751020703,iris_code_751020704,iris_code_751020801,iris_code_751020802,iris_code_751020803,iris_code_751020804,iris_code_751020805,iris_code_751030901,iris_code_751030902,iris_code_751030903,iris_code_751030904,iris_code_751030905,iris_code_751031001,iris_code_751031002,iris_code_751031003,iris_code_751031004,iris_code_751031101,iris_code_751031102,iris_code_751031103,iris_code_751031104,iris_code_751031201,iris_code_751031202,iris_code_751031203,iris_code_751031204,iris_code_751041301,iris_code_751041302,iris_code_751041303,iris_code_751041304,iris_code_751041401,iris_code_751041402,iris_code_751041403,iris_code_751041404,iris_code_751041501,iris_code_751041502,iris_code_751041503,iris_code_751041504,iris_code_751041601,iris_code_751041602,iris_code_751051701,iris_code_751051702,iris_code_751051703,iris_code_751051704,iris_code_751051705,iris_code_751051801,iris_code_751051802,iris_code_751051803,iris_code_751051804,iris_code_751051805,iris_code_751051806,iris_code_751051807,iris_code_751051808,iris_code_751051901,iris_code_751051902,iris_code_751051903,iris_code_751051904,iris_code_751051905,iris_code_751051906,iris_code_751051907,iris_code_751051908,iris_code_751052001,iris_code_751052002,iris_code_751052003,iris_code_751052004,iris_code_751052005,iris_code_751062101,iris_code_751062102,iris_code_751062103,iris_code_751062202,iris_code_751062203,iris_code_751062204,iris_code_751062301,iris_code_751062302,iris_code_751062303,iris_code_751062304,iris_code_751062305,iris_code_751062306,iris_code_751062307,iris_code_751062308,iris_code_751062309,iris_code_751062310,iris_code_751062311,iris_code_751062401,iris_code_751062402,iris_code_751072501,iris_code_751072502,iris_code_751072503,iris_code_751072504,iris_code_751072505,iris_code_751072506,iris_code_751072601,iris_code_751072602,iris_code_751072603,iris_code_751072604,iris_code_751072701,iris_code_751072702,iris_code_751072703,iris_code_751072704,iris_code_751072705,iris_code_751072706,iris_code_751072801,iris_code_751072802,iris_code_751072803,iris_code_751072804,iris_code_751072805,iris_code_751072806,iris_code_751072807,iris_code_751072808,iris_code_751072809,iris_code_751072810,iris_code_751072811,iris_code_751082901,iris_code_751082902,iris_code_751082903,iris_code_751082904,iris_code_751082905,iris_code_751083001,iris_code_751083002,iris_code_751083003,iris_code_751083004,iris_code_751083005,iris_code_751083006,iris_code_751083101,iris_code_751083102,iris_code_751083103,iris_code_751083104,iris_code_751083105,iris_code_751083106,iris_code_751083107,iris_code_751083108,iris_code_751083201,iris_code_751083202,iris_code_751083203,iris_code_751083204,iris_code_751083205,iris_code_751083206,iris_code_751083207,iris_code_751083208,iris_code_751083209,iris_code_751093301,iris_code_751093302,iris_code_751093303,iris_code_751093304,i

In [199]:
# Définition du dataset
# Sélectionner Paris sur les appartements (2) + sélection sur les quantiles
df_model = filter_quantile(df[df.code_type_local == 2], 'Prix_m2', 0.1, 0.99)

    # Définition des var explicatives
features_OLS_hors_geotemp = ['nombre_pieces_principales_dummy', 'Nombre_dependance_dummy', 'Vente_par_iris_tri',
               'IPL_{Q-1}', 'N_proprietaire', 'N_vacant', 'N_apparts']

# Pour les features geo : col_arrondissement OU col_iris
feature_geo = col_arrondissement

# Pour les features geo : col_month OU col_trimestre
feature_temp = col_month

    # Def de l'output
label = 'Prix_m2'

In [200]:
# Split des données en TRAIN et TEST
features_OLS = features_OLS_hors_geotemp + feature_geo + feature_temp
X_train, X_test, y_train, y_test = split_temporel_V1(df_model, features_OLS, label)

In [201]:
X_train.head(4)

,nombre_pieces_principales_dummy,Nombre_dependance_dummy,Vente_par_iris_tri,IPL_{Q-1},N_proprietaire,N_vacant,N_apparts,Arrondissement_75102,Arrondissement_75103,Arrondissement_75104,Arrondissement_75105,Arrondissement_75106,Arrondissement_75107,Arrondissement_75108,Arrondissement_75109,Arrondissement_75110,Arrondissement_75111,Arrondissement_75112,Arrondissement_75113,Arrondissement_75114,Arrondissement_75115,Arrondissement_75116,Arrondissement_75117,Arrondissement_75118,Arrondissement_75119,Arrondissement_75120,Month_10,Month_11,Month_12,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9
74630,1,0,27,103.5,565.0,142.0,1575.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
95474,0,0,11,103.5,624.0,145.0,1530.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
84819,0,0,9,103.5,460.0,100.0,1425.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
84827,1,0,16,103.5,596.0,179.0,1989.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [202]:
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [203]:
model = sm.OLS(y_train, X_train).fit()

In [204]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                Prix_m2   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1465.
Date:                Wed, 15 Mar 2023   Prob (F-statistic):               0.00
Time:                        11:09:25   Log-Likelihood:            -9.0398e+05
No. Observations:              102410   AIC:                         1.808e+06
Df Residuals:                  102372   BIC:                         1.808e+06
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [209]:
compute_scores(model, X_test, y_test)

MAPE:  15.79353842064714
MDAPE:  12.355083951933986


## OLS en modifiant les variables géo et temp

In [ ]:
    # Définition des var explicatives
features_OLS_hors_geotemp = ['nombre_pieces_principales_dummy', 'Nombre_dependance_dummy', 'Vente_par_iris_tri',
               'IPL_{Q-1}', 'N_proprietaire', 'N_vacant', 'N_apparts']

    # Def de l'output
label = 'Prix_m2'

In [ ]:
# Pour les features geo : col_arrondissement OU col_iris
feature_geo = col_arrondissement

# Pour les features geo : col_month OU col_trimestre
feature_temp = col_month

In [223]:
model_OLS(data = filter_quantile(df[df.code_type_local == 2], 'Prix_m2', 0.1, 0.99),
    feature_geo = feature_geo,
    feature_temp = feature_temp,
    feature_autre = features_OLS_hors_geotemp,
    output = label)

                            OLS Regression Results                            
Dep. Variable:                Prix_m2   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1465.
Date:                Wed, 15 Mar 2023   Prob (F-statistic):               0.00
Time:                        11:58:03   Log-Likelihood:            -9.0398e+05
No. Observations:              102410   AIC:                         1.808e+06
Df Residuals:                  102372   BIC:                         1.808e+06
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [224]:
%run '../Helpers/model_implementation.py'

In [225]:
# Pour les features geo : col_arrondissement OU col_iris
feature_geo = col_iris

# Pour les features geo : col_month OU col_trimestre
feature_temp = col_month

In [226]:
model_OLS(data = filter_quantile(df[df.code_type_local == 2], 'Prix_m2', 0.1, 0.99),
    feature_geo = feature_geo,
    feature_temp = feature_temp,
    feature_autre = features_OLS_hors_geotemp,
    output = label)

                            OLS Regression Results                            
Dep. Variable:                Prix_m2   R-squared:                       0.444
Model:                            OLS   Adj. R-squared:                  0.439
Method:                 Least Squares   F-statistic:                     90.91
Date:                Wed, 15 Mar 2023   Prob (F-statistic):               0.00
Time:                        12:01:15   Log-Likelihood:            -8.9573e+05
No. Observations:              102410   AIC:                         1.793e+06
Df Residuals:                  101519   BIC:                         1.802e+06
Df Model:                         890                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [227]:
# Pour les features geo : col_arrondissement OU col_iris
feature_geo = col_iris

# Pour les features geo : col_month OU col_trimestre
feature_temp = col_trimestre

model_OLS(data = filter_quantile(df[df.code_type_local == 2], 'Prix_m2', 0.1, 0.99),
    feature_geo = feature_geo,
    feature_temp = feature_temp,
    feature_autre = features_OLS_hors_geotemp,
    output = label)

                            OLS Regression Results                            
Dep. Variable:                Prix_m2   R-squared:                       0.443
Model:                            OLS   Adj. R-squared:                  0.438
Method:                 Least Squares   F-statistic:                     91.49
Date:                Wed, 15 Mar 2023   Prob (F-statistic):               0.00
Time:                        12:03:11   Log-Likelihood:            -8.9579e+05
No. Observations:              102410   AIC:                         1.793e+06
Df Residuals:                  101527   BIC:                         1.802e+06
Df Model:                         882                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     